In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [1]:
%load_ext autoreload

In [25]:
import pandas as pd
from tqdm import tqdm
import torch
import os
import numpy as np
from itertools import product

In [3]:
%autoreload 2
from huggingmolecules import MatConfig, MatFeaturizer, MatModel
from huggingmolecules import GroverConfig, GroverFeaturizer, GroverModel
from huggingmolecules import RMatConfig, RMatFeaturizer, RMatModel
from models import StrippedMatModel, StrippedGroverModel, StrippedRMatModel

In [19]:
%autoreload 2
from utils import *
from datasets import *
import predict_procedure as pp
import embedding_procedure as ep

In [36]:
%%bash

python embedding_procedure.py \
    --model_name Transformer-M \
    --dataset_name HIA_Hou \
    --max_samples 10

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'
Found local copy...
Loading...
Done!
Found local copy...


Processing HIA_Hou with Transformer-M


INFO: Processing HIA_Hou with Transformer-M
100%|██████████| 10/10 [00:00<00:00, 76.12it/s]


Processing HIA_Hou with Transformer-M took 0.0m


INFO: Processing HIA_Hou with Transformer-M took 0.0m


In [34]:
pd.read_pickle("embeddings/HIA_Hou/Transformer-M.zip").embeddings[92]

tensor([-0.0029, -0.0147, -0.0151,  0.0026, -0.0331,  0.0073, -0.0352,  0.0092,
        -0.0042,  0.0150, -0.0350, -0.0230,  0.0463, -0.0163,  0.0155,  0.0019,
         0.0145,  0.0257,  0.0089, -0.0323, -0.0041, -0.0025,  0.0642,  0.0287,
         0.0246, -0.0088, -0.0383, -0.0108,  0.0408, -0.0138,  0.0043,  0.0070,
        -0.0169, -0.0293,  0.1852,  0.0073, -0.0061,  0.0393,  0.0030,  0.0558,
        -0.0250, -0.0234,  0.0035, -0.0035,  0.0266,  0.0227,  0.0206, -0.0100,
        -0.0263, -0.0398,  0.0046, -0.0135,  0.0093,  0.0095, -0.0247,  0.0339,
         0.0071, -0.0428, -0.0430,  0.0510, -0.0017, -0.0209, -0.0076, -0.0144,
         0.0274,  0.0279,  0.0129,  0.0085, -0.0046, -0.0519,  0.0194, -0.0037,
        -0.0019, -0.0279,  0.0120,  0.0317, -0.0032,  0.0082, -0.0108, -0.0141,
        -0.0038, -0.0087,  0.0019, -0.0279,  0.0246,  0.0370, -0.0394,  0.0140,
        -0.0148, -0.0485, -0.0155, -0.0277,  0.0066,  0.0097,  0.0304, -0.0374,
        -0.0092, -0.0368, -0.0029, -0.00

In [6]:
# from mat import MolecularAttentionTransformer

# model = MolecularAttentionTransformer()
# model.predict(["CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1"]) ## tooks 11m 4s (!)

In [7]:
# group = admet_group(path = 'data/')
# benchmark = group.get("CYP2D6_Substrate_CarbonMangels")

# dataset = benchmark["test"]

# print(dataset[dataset.Drug == "CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C"].to_markdown())

# Output:
# |     | smiles                             |   Y |
# |----:|:-----------------------------------|----:|
# |  58 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   1 |
# | 226 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   0 |else 'regression'

In [37]:
import pathlib

calculated = set(map(str, pathlib.Path("embeddings/").glob("*/*.zip")))

pairs = product(get_dataset_names(), ["Transformer-M"])

with open("embedding_params.txt", "w") as f:
    for dataset_name, model_name in pairs:
        if f"embeddings/{dataset_name}/{model_name}.zip" not in calculated:
            print(dataset_name, model_name, file=f)


In [41]:
import pathlib

with open("predict_params.txt", "w") as f:

    done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

    pairs = list(product(get_regression_names(), ["Transformer-M"]))
    calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

    params = list(product(list(set(pairs)), ["linear", "forest", "ridge"]))


    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" not in done:
            print(dataset_name, model_name, head_model, file=f)

    pairs = list(product(get_classification_names(), ["Transformer-M"]))
    calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

    params = list(product(list(set(pairs)), ["linear", "forest"]))


    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" not in done:
            print(dataset_name, model_name, head_model, file=f)
            

In [36]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_dataset_names(), get_embedder_names()))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs) & set(calculated)), ["ridge","forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" in done and dataset_name in get_classification_names():
            print(dataset_name, model_name, head_model, file=f)

In [ ]:
with open("selformer_params.txt", "w") as f:
    for dataset_name in get_dataset_names():
        if len(get_dataset(dataset_name)[0]) < 10000:
            f.write(f"{dataset_name}\n")
    

In [68]:
from tdc.chem_utils import MolConvert
converter = MolConvert(src = 'SMILES', dst = 'PyG')
converter('C').x
# Data(edge_index=[2, 68], edge_attr=[68, 3], x=[32, 9], y=[1, 1], num_nodes=32)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 0.]])

In [69]:
from ogb.utils import smiles2graph
{**smiles2graph('C'), "x": converter('C').x, "y": np.array([0])}

{'edge_index': array([], shape=(2, 0), dtype=int64),
 'edge_feat': array([], shape=(0, 3), dtype=int64),
 'node_feat': array([[5, 0, 4, 5, 4, 0, 2, 0, 0]]),
 'num_nodes': 1,
 'x': tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
          0., 0., 0.]]),
 'y': array([0])}

In [64]:
get_dataset("HIA_Hou", True)[0]

Found local copy...
Loading...
Done!
Found local copy...


,smiles,Y,graph
0,CC(=O)Nc1ccccc1,1,"{'edge_index': [[0, 1, 1, 2, 1, 3, 3, 4, 4, 5,..."
1,CC(=O)Nc1nnc(S(N)(=O)=O)s1,1,"{'edge_index': [[0, 1, 1, 2, 1, 3, 3, 4, 4, 5,..."
2,Cc1c(N(C)C)c(=N)n(-c2ccccc2)n1C,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 3, 5,..."
3,COc1ccccc1OCCNC[C@@H](O)c1ccc(C)cc1S(N)(=O)=O,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5,..."
4,NS(=O)(=O)c1cc2c(cc1C(F)(F)F)N[C@H](Cc1ccccc1)...,1,"{'edge_index': [[0, 1, 1, 2, 1, 3, 1, 4, 4, 5,..."
...,...,...,...
573,CC(=CCO)/C=C/C=C(C)\C=C\C1=C(C)CCCC1(C)C,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 1, 5,..."
574,CCN(CC)[C@H](C)C(=O)c1ccccc1,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 2, 5,..."
575,OC[C@@H]1O[C@@H](OC[C@@H]2O[C@@H](O[C@]3(CO)O[...,0,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5,..."
576,COC(=O)C1=C(C#N)NC(C)=C(C(=O)OC(C)C)[C@H]1c1cc...,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 2, 4, 4, 5,..."


In [5]:
from pretrain_gnns.chem.model import GNN_graphpred

In [11]:
model = GNN_graphpred(5, 300, 1)
model.from_pretrained("pretrain_gnns/chem/model_gin/supervised_masking.pth")
model.graph_pred_linear = torch.nn.Identity()
model

GNN_graphpred(
  (gnn): GNN(
    (x_embedding1): Embedding(120, 300)
    (x_embedding2): Embedding(3, 300)
    (gnns): ModuleList(
      (0-4): 5 x GINConv()
    )
    (batch_norms): ModuleList(
      (0-4): 5 x BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (graph_pred_linear): Identity()
)

In [12]:
from rdkit import Chem
from pretrain_gnns.chem.loader import mol_to_graph_data_obj_simple

a = mol_to_graph_data_obj_simple(Chem.MolFromSmiles('CC'))
a.id = torch.tensor([0])
a.y = torch.tensor([0])
a

Data(x=[2, 2], edge_index=[2, 2], edge_attr=[2, 2], id=[1], y=[1])

In [16]:
model(a).flatten()

tensor([-2.2860e-03, -1.0651e-02, -1.3935e-02, -1.2835e-04, -3.0596e-02,
         6.1413e-03, -3.4948e-02,  1.1027e-02, -8.0211e-03,  1.5390e-02,
        -4.1179e-02, -2.1974e-02,  4.7206e-02, -1.6192e-02,  1.6484e-02,
         2.8944e-03,  9.4745e-03,  2.7458e-02,  8.4795e-03, -3.2276e-02,
        -3.4757e-03, -2.4873e-03,  7.0789e-02,  2.8699e-02,  2.9516e-02,
        -9.0462e-03, -3.9536e-02, -9.0948e-03,  3.7251e-02, -1.3832e-02,
         4.4269e-03,  1.4694e-02, -1.4983e-02, -2.7421e-02,  1.8085e-01,
         1.0816e-02, -5.2186e-03,  3.9262e-02,  2.4130e-03,  5.4883e-02,
        -2.5037e-02, -1.9981e-02,  3.5048e-03, -2.3913e-03,  2.6480e-02,
         2.2530e-02,  2.0608e-02, -1.2867e-02, -2.5237e-02, -4.2520e-02,
         1.2864e-03, -1.3701e-02,  1.0453e-02,  9.5401e-03, -2.4706e-02,
         3.3749e-02,  6.7488e-03, -4.1012e-02, -4.2344e-02,  5.3390e-02,
        -1.6355e-03, -2.1328e-02, -7.5259e-03, -1.2541e-02,  2.6705e-02,
         2.7715e-02,  1.2930e-02,  9.7416e-03, -4.6